<a href="https://colab.research.google.com/github/gianmarco-holm/PY01_MachineLearning_LinearRegression_HousePrices/blob/main/PY01_MachineLearning_LinearRegression_HousePrices_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Regresión Lineal de Precios de Casa

Este proyecto utiliza técnicas de machine learning, específicamente un modelo de regresión lineal, para predecir los precios de las casas en Boston utilizando el conjunto de datos del Boston Housing Dataset. El objetivo es desarrollar un modelo que pueda proporcionar estimaciones precisas del valor mediano de las viviendas ocupadas por sus propietarios basándose en diversas características socioeconómicas y ambientales del entorno.

## Carga de Datos

In [ ]:
# Este comando solo se ejecuta una vez para actualizar pip y pueda usarse la librería stats, luego se vuelve a comentar
#!pip install --upgrade pip setuptools==57.5.0

In [ ]:
# Comando que nos permite instalar la librería que tiene a stats
!pip install regressors

In [ ]:
# Importar librerías

# Librerías de transformación
import pandas as pd
import numpy as np

# Librerías de visualización
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Librerías de ML
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from regressors import stats
import pickle

In [ ]:
# Asignar estilo a seaborn
sns.set(style = 'darkgrid', context = 'notebook')

In [ ]:
# Cargar datos
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data', header=None, sep='\s+')
df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


**Definición funcional:**

* CRIM: Tasa de crimen per cápita por pueblo.
* ZN: Proporción de terreno residencial zonificado para lotes de más de 25,000 pies cuadrados.
* INDUS: Proporción de acres de negocios no minoristas por pueblo.
* CHAS: Variable ficticia del río Charles (1 si la parcela limita con el río; 0 en caso contrario).
* NOX: Concentración de óxidos de nitrógeno (partes por 10 millones).
* RM: Número promedio de habitaciones por vivienda.
* AGE: Proporción de unidades ocupadas por el propietario construidas antes de 1940.
* DIS: Distancias ponderadas a cinco centros de empleo de Boston.
* RAD: Índice de accesibilidad a carreteras radiales.
* TAX: Tasa de impuesto a la propiedad de valor completo.
* PTRATIO: Proporción de alumnos por maestro por pueblo.
* LSTAT: % de población de estatus social más bajo.
* MEDV: Valor mediano de las viviendas ocupadas por el propietario en $1000.

## Análsis Exploratior de Datos

In [ ]:
# Seleccionar columnas a analizar
cols = ['DIS', 'INDUS', 'CRIM', 'RM', 'MEDV']
# Crear una matriz de dispersion para analizar las columnas
fig = px.scatter_matrix(df[cols], title="Matriz de Dispersión", opacity=0.5)
fig.show()

**Conclusión 01:**
* Podemos observar que hay una correación entre RM y MEDV
* Además vemos que Indus se divide en dos sectores
* Haremos ahora la matriz de correlación para ver númericamente estas relaciones en las variables

In [ ]:
# Crear la correlación de variables
matriz_correlacion = df[cols].corr()

In [ ]:
# Crear el heatmap usando imshow
fig = px.imshow(
    matriz_correlacion,
    text_auto=True,
    aspect="auto",
    color_continuous_scale='Viridis',
    title='Heatmap de la Matriz de Correlación'
)

# Mostrar el gráfico
fig.show()

**Conclusión 02:**
* Vemos que hay bastante correlación entre MEDV y RM, lo que nos indica que a mayor número de cuartos, mayor es el precio de la casa.
* Existe una correlación en sentido negativo entre CRIM y MEDV, lo que significa que ha mayor indice de criminalidad menor el precio de la casa.
* Tambien vemos que hay una correlación entre DIS e INDUS, lo que significa que a mayor negocios menos distancia desde la casa hasta el empleo.

> Debido a estas conclusiones utilizaremos estas 4 variables como independientes y probaremos el modelo hasta calcular el valor p y verificar si estas variables son convenientes o no para el modelo.

## Entrenar Modelo de Datos

In [ ]:
# Separar la variable independiente y dependiente
X_cols = ['DIS', 'INDUS', 'CRIM', 'RM']
y_cols = 'MEDV'

# Dividir los datos en conjuntos de entrenamiento y de prueba
X_train, X_test, y_train, y_test = train_test_split(df[X_cols], df[y_cols], test_size=0.2, random_state=42)

# Escalar datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Ajustamos las estadísticas y transformamos
X_test_scaled = scaler.transform(X_test) # Con las estadísticas ya ajustado, aca solo trasnformamos

# Crear el modelo
model = LinearRegression()
model.fit(X_train_scaled, y_train) # Entrenamos el modelo

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Crear un DataFrame con los resultados
results_df = X_test.copy()
results_df['Valores Reales'] = y_test.values
results_df['Valores Predichos'] = y_pred
results_df

,DIS,INDUS,CRIM,RM,Valores Reales,Valores Predichos
173,2.6463,4.05,0.09178,6.416,23.6,27.284267
274,4.0776,6.41,0.05644,6.758,32.4,28.399723
491,1.8681,27.74,0.10574,5.983,13.6,16.636973
72,5.2873,10.81,0.09164,6.065,22.8,20.815287
452,2.3682,18.10,5.09017,6.297,16.1,20.835634
...,...,...,...,...,...,...
412,1.5539,18.10,18.81100,4.628,17.9,5.097497
436,2.0026,18.10,14.42080,6.461,9.6,20.253676
411,1.5275,18.10,14.05070,6.657,17.2,22.151894
86,4.4272,4.49,0.05188,6.015,22.5,22.972393


## Evaluar el modelo

In [ ]:
# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Error cuadrático medio (MSE): {mse:.2f}')
print(f'Coeficiente de determinación (R^2): {r2:.2f}')

Error cuadrático medio (MSE): 36.66
Coeficiente de determinación (R^2): 0.50


**Interpretación:**

1. Error Cuadrático Medio (MSE): Este valor indica en promedio cuánto se desvían las predicciones del modelo de los valores reales. En este caso, un MSE de 36.66 sugiere que las predicciones tienen un error medio cuadrático de aproximadamente 36.66 unidades al cuadrado en la escala de la variable dependiente (en este caso, el precio de las viviendas MEDV). Esto significa que si el valor real es 20.00, este tiende a desviarse la raiz cuadrada de 36.66, es decir +-6.

2. Coeficiente de Determinación (𝑅2): Este coeficiente proporciona una medida de cuánta variabilidad en los datos es explicada por el modelo. Un R2 de 0.50 significa que aproximadamente el 50% de la variabilidad en los precios de las viviendas (MEDV) puede ser explicada por las características utilizadas en el modelo (DIS, INDUS, CRIM, RM). En otras palabras, el modelo explica la mitad de la variabilidad observada en los precios de las viviendas.

In [ ]:
# Calcular estadísticas para evaluar el modelo a mayor detalle
print('========= Summary =========')
stats.summary(model, X_test_scaled, y_test, X_cols)

# Validación cruzada
cv_scores = cross_val_score(model,X_train_scaled, y_train, cv=10, scoring='r2')
print(f'Validación cruzada R²: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}')

========= Summary =========
Residuals:
     Min      1Q  Median     3Q     Max
-30.0396 -2.1006   0.445 2.8219 11.1976


Coefficients:
             Estimate  Std. Error  t value   p value
_intercept  22.796535    0.613755  37.1427  0.000000
DIS         -1.219844    0.863591  -1.4125  0.160869
INDUS       -2.232566    0.849706  -2.6275  0.009943
CRIM        -1.978687    0.816328  -2.4239  0.017134
RM           5.579410    0.685109   8.1438  0.000000
---
R-squared:  0.50010,    Adjusted R-squared:  0.47948
F-statistic: 24.26 on 4 features
Validación cruzada R²: 0.54 ± 0.24


**Conclusión 03:**
* Podemos observar que DIS tiene un valor p mayor a 0.05 lo que significa que no suma en el modelo, por lo que procederemos a quitar esta variable

## Evaluación del modelo 2

In [ ]:
# Separar la variable independiente y dependiente
X2_cols = ['INDUS', 'CRIM', 'RM']
y2_cols = 'MEDV'

df2 = df.copy()

# Dividir los datos en conjuntos de entrenamiento y de prueba
X2_train, X2_test, y2_train, y2_test = train_test_split(df2[X2_cols], df2[y2_cols], test_size=0.2, random_state=42)

# Escalar datos
scaler2 = StandardScaler()
X2_train_scaled = scaler2.fit_transform(X2_train) # Ajustamos las estadísticas y transformamos
X2_test_scaled = scaler2.transform(X2_test) # Con las estadísticas ya ajustado, aca solo trasnformamos

# Crear el modelo
model2 = LinearRegression()
model2.fit(X2_train_scaled, y2_train) # Entrenamos el modelo

# Hacer predicciones en el conjunto de prueba
y2_pred = model2.predict(X2_test_scaled)

# Crear un DataFrame con los resultados
results_df2 = X2_test.copy()
results_df2['Valores Reales'] = y2_test.values
results_df2['Valores Predichos'] = y2_pred
results_df2

,INDUS,CRIM,RM,Valores Reales,Valores Predichos
173,4.05,0.09178,6.416,23.6,25.716712
274,6.41,0.05644,6.758,32.4,28.009355
491,27.74,0.10574,5.983,13.6,17.514993
72,10.81,0.09164,6.065,22.8,21.545285
452,18.10,5.09017,6.297,16.1,20.920069
...,...,...,...,...,...
412,18.10,18.81100,4.628,17.9,4.611390
436,18.10,14.42080,6.461,9.6,20.292064
411,18.10,14.05070,6.657,17.2,21.948026
86,4.49,0.05188,6.015,22.5,22.407737


In [ ]:
# Evaluar el modelo
mse = mean_squared_error(y2_test, y2_pred)
r2 = r2_score(y2_test, y2_pred)
print(f'Error cuadrático medio (MSE): {mse:.2f}')
print(f'Coeficiente de determinación (R^2): {r2:.2f}')

Error cuadrático medio (MSE): 37.84
Coeficiente de determinación (R^2): 0.48


In [ ]:
# Calcular estadísticas para evaluar el modelo a mayor detalle
print('========= Summary =========')
stats.summary(model2, X2_test_scaled, y2_test, X2_cols)

# Validación cruzada
cv_scores2 = cross_val_score(model2,X2_train_scaled, y2_train, cv=10, scoring='r2')
print(f'Validación cruzada R²: {cv_scores2.mean():.2f} ± {cv_scores2.std():.2f}')

========= Summary =========
Residuals:
     Min      1Q  Median     3Q     Max
-30.5975 -2.1384  0.6795 3.1023 11.0707


Coefficients:
             Estimate  Std. Error  t value   p value
_intercept  22.796535    0.623271  36.5756  0.000000
INDUS       -1.369442    0.697322  -1.9639  0.052295
CRIM        -1.851471    0.808389  -2.2903  0.024082
RM           5.707117    0.695178   8.2096  0.000000
---
R-squared:  0.48402,    Adjusted R-squared:  0.46823
F-statistic: 30.64 on 3 features
Validación cruzada R²: 0.53 ± 0.24


Como vimos el segundo modelo tiene un R2 de 48%, que es menor al primer modelo de 50%, por lo cual nos quedaremos con el primer modelo y lo analizaremos a nivel de gráfico

## Evaluación Visual del modelo

In [ ]:
# Grafico de dispersión: Valores Reales vs. Predichos
fig_scatter = px.scatter(results_df, x='Valores Reales', y='Valores Predichos', title='Valores Reales vs. Predichos')
fig_scatter.add_shape(type='line',
                      x0=results_df['Valores Reales'].min(),
                      y0=results_df['Valores Reales'].min(),
                      x1=results_df['Valores Reales'].max(),
                      y1=results_df['Valores Reales'].max(),
                      line=dict(color='Red', dash='dash'))
fig_scatter.show()

**Interpretación:**
1. La recta roja es la linea de identidad y es cuando X=Y.
2. Esto significa que mientras que los puntos esten mas concentrados en la linea de identidad mas preciso es el modelo.
3. Tambien nos permite observar que los valores predichos encima de la recta estan sobreestimados, y las que estan debajo estan subestimados. En este caso vemos que hay mas outliers que estan subestimando los precios reales.

In [ ]:
# Gráfico de residuos
residuals = y_test - y_pred
fig_residuals = px.scatter(x=y_pred, y=residuals, title='Gráfico de Residuos')
fig_residuals.add_shape(type='line', x0=y_pred.min(), y0=0, x1=y_pred.max(), y1=0,
                        line=dict(color='Red', dash='dash'))
fig_residuals.update_layout(
    xaxis_title="Valores Predichos",
    yaxis_title="Residuos"
)
fig_residuals.show()

**Interpretación:**
1. La linea roja indica cuando el residuo es 0
2. Esto significa que si los puntos se concentran a la linea roja, el modelo esta bien ajustado.
3. Los puntos que estan abajo y alejados de la linea son los valores predichos sobreestimados, y los que estan lejando arriba son los subestimados.

In [ ]:
# Seleccionar un subconjunto de datos para visualizar
num_samples = 30
indices = np.arange(num_samples)
real_values = y_test[:num_samples].reset_index(drop=True)
pred_values = y_pred[:num_samples]

# Crear el gráfico de barras
fig = go.Figure()
fig.add_trace(go.Bar(x=indices, y=real_values, name='Valores Reales'))
fig.add_trace(go.Bar(x=indices, y=pred_values, name='Valores Predichos'))

# Personalizar el gráfico
fig.update_layout(
    title='Comparación de Valores Reales y Predichos',
    xaxis=dict(title='Muestras'),
    yaxis=dict(title='Valores'),
    barmode='group'
)

fig.show()

**Interpretación:**
1. Aca cogemos las 30 primeras filas de los valores predichos y reales.
2. Donde podemos observar la diferencia que hay entre los valores reales y predichos, concluimos que hay 2 barras que tienen demasiada diferencia, sin embargo las demas estan bien.

## Guardar y cargar el modelo de Machine Learning

Si queremos guardar el modelo podemos usar este codigo

In [ ]:
# Guardar el modelo usando pickle
#with open('modelo_regresion_lineal.pkl', 'wb') as archivo:
#    pickle.dump(model, archivo)

In [ ]:
# Cargar el modelo usando pickle
#with open('modelo_regresion_lineal.pkl', 'rb') as archivo:
#    modelo_cargado = pickle.load(archivo)